<a href="https://colab.research.google.com/github/maximecharriere/CryptoWallet/blob/main/CryptoWallet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import libraries

In [3]:
import os
import requests
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
import json
# from google.colab import files
import io
import pandas as pd
import numpy as np
DEV_MODE = False

In [20]:
# create an example pandas dataframe
df = pd.DataFrame({'A': [1, 2, 3], 'B': [4, 5, 6], 'Sum': [np.nan, 17, 19]})

df.loc[df.Sum.isna(),'Sum'] = df[df.Sum.isna()].apply(lambda row: row['A'] + row['B'], axis=1)

print(df)

   A  B   Sum
0  1  4   5.0
1  2  5  17.0
2  3  6  19.0


## Binance

In [17]:
# TEST
data = pd.concat([
    pd.read_csv(r"C:\Users\conta\SynologyDrive\Documents\Crypto\Transactions\binance1_2021-03-01 - 2021-10-31.csv"),
    pd.read_csv(r"C:\Users\conta\SynologyDrive\Documents\Crypto\Transactions\binance1_2021-11-01 - 2022-10-20.csv"), 
    pd.read_csv(r"C:\Users\conta\SynologyDrive\Documents\Crypto\Transactions\binance2_2021-01-01 - 2021-12-31.csv"), 
    pd.read_csv(r"C:\Users\conta\SynologyDrive\Documents\Crypto\Transactions\binance2_2022-01-01 - 2022-10-25.csv")
    ], ignore_index=True)

print(data['Operation'].unique().sort())

None


### Import data
To generate history on [binance.com](binance.com) : Wallet -> Transaction History -> Generate all statements

In [ ]:
uploaded = files.upload()
assert len(uploaded)==1, "Only one file must be added"


### Replace coins names


In [ ]:
coinMap = {'ERD':'EGLD','SHIB2':'SHIB'}

for fn in uploaded.keys():
    data = pd.read_csv(io.BytesIO(uploaded[fn]))
    data['Coin'].replace(coinMap, inplace=True, regex=True) 


### Test if all operations are handle by the script

In [ ]:
for op in data['Operation'].unique():
    assert op in ['Buy',
                  'Sell',
                  'POS savings purchase',
                  'POS savings redemption',
                  'POS savings interest',
                  'ETH 2.0 Staking',
                  'ETH 2.0 Staking Rewards',
                  'Savings Interest',
                  'Savings purchase',
                  'Savings Principal redemption',
                  'Cash Voucher distribution',
                  'Distribution',
                  'Rewards Distribution',
                  'Deposit',
                  'Fee',
                  'Small assets exchange BNB',
                  'Transaction Related',
                  'Withdraw'], \
                  (f'The "{op}" operation is not currently handle by this script. '
                  'Create an Issue on the Github Repository to add this operation.')

### Extract data by "Operation" and group them by coins

In [ ]:
spotBuy             = data.loc[data['Operation'] == 'Buy'                         ].groupby('Coin')['Change'].sum()
spotSell            = data.loc[data['Operation'] == 'Sell'                        ].groupby('Coin')['Change'].sum()
stakingPurchase     = data.loc[data['Operation'] == 'POS savings purchase'        ].groupby('Coin')['Change'].sum()
stakingRedemption   = data.loc[data['Operation'] == 'POS savings redemption'      ].groupby('Coin')['Change'].sum()
stakingInterest     = data.loc[data['Operation'] == 'POS savings interest'        ].groupby('Coin')['Change'].sum()
stakingEth2Purchase = data.loc[data['Operation'] == 'ETH 2.0 Staking'             ].groupby('Coin')['Change'].sum()
stakingEth2Interest = data.loc[data['Operation'] == 'ETH 2.0 Staking Rewards'     ].groupby('Coin')['Change'].sum()
savingsInterest     = data.loc[data['Operation'] == 'Savings Interest'            ].groupby('Coin')['Change'].sum()
savingsPurchase     = data.loc[data['Operation'] == 'Savings purchase'            ].groupby('Coin')['Change'].sum()
savingsRedemption   = data.loc[data['Operation'] == 'Savings Principal redemption'].groupby('Coin')['Change'].sum()
cashDistribution    = data.loc[data['Operation'] == 'Cash Voucher distribution'   ].groupby('Coin')['Change'].sum()
distribution        = data.loc[data['Operation'] == 'Distribution'                ].groupby('Coin')['Change'].sum()
rewardsDistribution = data.loc[data['Operation'] == 'Rewards Distribution'        ].groupby('Coin')['Change'].sum()
deposit             = data.loc[data['Operation'] == 'Deposit'                     ].groupby('Coin')['Change'].sum()
fee                 = data.loc[data['Operation'] == 'Fee'                         ].groupby('Coin')['Change'].sum()
exchangeBNB         = data.loc[data['Operation'] == 'Small assets exchange BNB'   ].groupby('Coin')['Change'].sum()
transaction         = data.loc[data['Operation'] == 'Transaction Related'         ].groupby('Coin')['Change'].sum()
withdraw            = data.loc[data['Operation'] == 'Withdraw'                    ].groupby('Coin')['Change'].sum()


### Group operations
Group operation by wallet
- Spot
- Savings + Savings Interest
- Stacking + Stacking Interest


In [ ]:
# Remove fiat
fiat = ['USD', 'GBP', 'SEK', 'CAD', 'EUR', 'CNY', 'RUB', 'TRY', 'NGN', 'UAH', 
        'KZT', 'VND', 'COH', 'PLN', 'INR', 'BRL', 'ARS', 'MXN', 'IDR']
data = data[~data.Coin.isin(fiat)]


result = {}
result['Spot'] = data.groupby('Coin')['Change'].sum()
result['Savings'] = result['Spot'].loc[result['Spot'].index.str.startswith('LD')].rename(index=lambda s: s[2:]) # Coin begining with LD are coin stored in savings wallet
result['Spot'].drop(result['Spot'].index[result['Spot'].index.str.startswith('LD')],inplace=True) # Remove coins found in saving wallet (beginning with LD) from the spot wallet
result['Savings Interest'] = savingsInterest.add(rewardsDistribution,fill_value=0)
result['Staking'] = (-stakingPurchase).sub(stakingRedemption,fill_value=0)
result['Staking Interest'] = stakingInterest.add(stakingEth2Interest,fill_value=0)

result['Staking'] = result['Staking'].add(result['Spot'].loc[result['Spot'].index=='BETH'],fill_value=0) # Move ETH2.0 stacking coin BETH from spot to staking wallet
result['Spot'].drop('BETH',inplace=True) # Remove coin BETH from spot

resultDf = pd.DataFrame(result).fillna(0)

## Add USD value

In [ ]:
def addUsdValue(df, colName, currentPriceJson):
    newColIdx = df.columns.get_loc(colName) + 1
    newColName = colName + ' (USD)'
    newColData = df.apply(lambda row : currentPriceJson['data'][row.name]["quote"]["USD"]["price"]*row[colName], axis=1)
    df.insert(newColIdx, newColName, newColData)


In [ ]:
# Config CMC API

if DEV_MODE:
    api_url = 'https://sandbox-api.coinmarketcap.com/v1/cryptocurrency/quotes/latest'
    api_headers = {
    'Accepts': 'application/json',
    'X-CMC_PRO_API_KEY': 'b54bcf4d-1bca-4e8e-9a24-22ff2c3d462c',
    }
else:
    api_url = 'https://pro-api.coinmarketcap.com/v1/cryptocurrency/quotes/latest'
    api_headers = {
      'Accepts': 'application/json',
      'X-CMC_PRO_API_KEY': 'b1500d10-1275-47e2-82b7-f722cd05398a',
    }


api_parameters = {
  'symbol':"",
  'convert':'USD'
}   
api_session = requests.Session()
api_session.headers.update(api_headers)

# Modify symbol name different in CMC
symbol_map = {'IOTA':'MIOTA'}
resultDf.rename(index = symbol_map, inplace=True)

# Create symbol list
api_parameters['symbol'] = ','.join(resultDf.index.values)

# Get USD prices
try:
  response = api_session.get(api_url, params=api_parameters)
  response_json = json.loads(response.text)
except (ConnectionError, Timeout, TooManyRedirects) as e:
  print(e)

assert response_json['status']['error_code'] == 0,\
 (f"CoinMarketCap Error N°{response_json['status']['error_code']} : {response_json['status']['error_message']}")

# Add USD value
addUsdValue(resultDf,'Spot',response_json)
addUsdValue(resultDf,'Savings',response_json)
addUsdValue(resultDf,'Staking',response_json)
resultDf['Tot (USD)'] = resultDf.apply(lambda row : row['Spot (USD)'] + row['Savings (USD)'] + row['Staking (USD)'], axis=1)


## Print result

In [ ]:
print(f"Tot Spot :    {resultDf['Spot (USD)'].sum():.2f} (USD)")
print(f"Tot Savings : {resultDf['Savings (USD)'].sum():.2f} (USD)")
print(f"Tot Staking : {resultDf['Staking (USD)'].sum():.2f} (USD)")
print("____________________________")
print(f"Tot :         {resultDf['Tot (USD)'].sum():.2f} (USD)")

resultDf

## Export result

In [ ]:
resultDf.to_csv('output.csv')
files.download('output.csv')

## Test CryptoCompare API


In [5]:

api_url = 'https://min-api.cryptocompare.com/data/v2/histohour'

api_parameters = {
  'fsym':"ADA",
  'tsym':'USD',
  'limit':'1',
  'toTs':'1664440522',
  'extraParams':'CryptoWallet'
}   
api_session = requests.Session()
# api_session.headers.update(api_headers)

# Get USD prices
try:
  response = api_session.get(api_url, params=api_parameters)
  response_json = json.loads(response.text)
except (ConnectionError, Timeout, TooManyRedirects) as e:
  print(e)
  raise

assert response_json['Response'] == "Success",\
 (f"CryptoCompare API Error : {response_json['Message']}")

print(json.dumps(response_json, indent=2))
data = response_json['Data']['Data'][-1]
mean_price = (data['high'] + data['low'])/2
print(f"Price: {mean_price:.7} (USD)")

{
  "Response": "Success",
  "Message": "",
  "HasWarning": false,
  "Type": 100,
  "RateLimit": {},
  "Data": {
    "Aggregated": false,
    "TimeFrom": 1664434800,
    "TimeTo": 1664438400,
    "Data": [
      {
        "time": 1664434800,
        "high": 0.433,
        "low": 0.4296,
        "open": 0.4325,
        "volumefrom": 3790813.83,
        "volumeto": 1633719.45,
        "close": 0.4314,
        "conversionType": "direct",
        "conversionSymbol": ""
      },
      {
        "time": 1664438400,
        "high": 0.4343,
        "low": 0.4309,
        "open": 0.4314,
        "volumefrom": 1640247.58,
        "volumeto": 709696.31,
        "close": 0.4341,
        "conversionType": "direct",
        "conversionSymbol": ""
      }
    ]
  }
}
Price: 0.4326 (USD)


In [5]:
import random
import time 
import pandas as pd

def slowRandom(min:int, max:int):
    time.sleep(1)
    return random.randint(min,max)

dataframe = pd.DataFrame({'Min':[1,2,3],'Max':[11, 12, 13],'Rand':[pd.NA,pd.NA,pd.NA]})

dataframe['Rand'] = dataframe.apply(lambda row : slowRandom(row['Min'],row['Max']), axis=1)

dataframe



,Min,Max,Rand
0,2,10,6
1,7,18,16
2,4,12,7
